# How to use the invenio API

- first run:

  ```bash
  export PYTHON=/usr/bin/python3.12
  ./nrp upgrade
  ./nrp develop
  ```

- run API with:

  ```bash
  ./run.sh
  ```

- grant user access (doesn't work):

  ```bash
  source .venv/bin/activate
  invenio access allow admin-access user test@test.com
  ```

## Useful links

- [Invenio API requests](https://invenio.readthedocs.io/en/latest/getting-started/quickstart/crud-operations.html)
- [Invenio REST API](https://inveniordm.docs.cern.ch/reference/rest_api_index/) (they are lying about authentication only with Access Tokens, you can authenticate as shown below)
- [NRP Docs](https://nrp-cz.github.io/docs)
- [NRP models](https://narodni-repozitar.github.io/developer-docs/docs/technology/invenio/nrp-toolchain/edit-metadata)

## Actions

In [1]:
import requests

# disable warnings for self-signed certs
requests.packages.urllib3.disable_warnings()

API_ENDPOINT = 'https://mdrepo.eu/api'
# API_ENDPOINT = 'https://inveniordm.web.cern.ch/api'

# test if API is up
response = requests.get(f'{API_ENDPOINT}/login')
if response.status_code != 405:
    raise Exception('API returned with code', response.status_code)

### Login

In [9]:
data = {
    'email': 'test@test.com',
    'password': '123456'
}

response = requests.post(
    f'{API_ENDPOINT}/login',
    data=data
)
print(response.json())

session_cookie = response.cookies['session']
print(response.cookies)
print('Session cookie:', session_cookie)

{'confirmed_at': '2024-10-05T12:29:49.522756', 'email': 'test@test.com', 'id': 1, 'last_login_at': '2024-10-05T12:36:13.691414', 'roles': []}
<RequestsCookieJar[<Cookie session=2d96cef23814fe5f_670133f3.dyMwzMEa9INOHwQuoM8kOjU6Rnw for mdrepo.eu/>]>
Session cookie: 2d96cef23814fe5f_670133f3.dyMwzMEa9INOHwQuoM8kOjU6Rnw


### Search

In [ ]:
params = {
    'q': 'test',
    'size': 10,
    'page': 1
}
response = requests.get(f'{API_ENDPOINT}/records', params=params)
print(response.json())

### Create record

In [8]:
# XXX: session_cookie is required to create a record

cookies = {'session': session_cookie}
data = {'title':'Cool title', 'contributors': [{'name': 'Doe, John'}]}

response = requests.post(
    f'{API_ENDPOINT}/records',
    cookies=cookies,
    data=data
)
response.raise_for_status()

print(response.json())

if 'errors' in response.json():
    print(response.json()['errors'])

record_id = response.json().get('id', None)
print('Record ID:', record_id)

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://mdrepo.eu/api/records/

### View record

In [ ]:
response = requests.get(f'{API_ENDPOINT}/records/{record_id}')
print(response.json())

### List files of a record

In [ ]:
response = requests.get(f'{API_ENDPOINT}/records/{record_id}/files')
print(response.json())

### Upload file to a record

In [ ]:
local_file_name = 'resp1.json'
remote_file_name = 'search_response.json'

headers = {
    'Referer': API_ENDPOINT,
    'X-CSRFToken': csrf_token
}
cookies = {
    'csrftoken': csrf_token,
    'session': session_cookie
}

# set file metadata
data = [{'key': remote_file_name}]
requests.post(f'{API_ENDPOINT}/records/{record_id}/draft/files', headers=headers, cookies=cookies, json=data)
print('POST metadata:', response.json())

# upload file
with open(local_file_name, 'rb') as f:
    response = requests.put(
        f'{API_ENDPOINT}/records/{record_id}/draft/files/{remote_file_name}/content',
        headers=headers,
        cookies=cookies,
        data=f
    )
    print('PUT file:', response.json())

# commit file
response = requests.post(
    f'{API_ENDPOINT}/records/{record_id}/draft/files/{remote_file_name}/commit',
    headers=headers,
    cookies=cookies
)
print('POST commit:', response.json())


### Download file from a record

In [ ]:
remote_file_name = '20240802_151338.jpg'
response = requests.get(f'{API_ENDPOINT}/records/{record_id}/files/{remote_file_name}/content')

with open(remote_file_name, 'wb') as f:
    f.write(response.content)

### Publish record

In [ ]:
headers = {
    'Referer': API_ENDPOINT,
    'X-CSRFToken': csrf_token
}
cookies = {
    'csrftoken': csrf_token,
    'session': session_cookie
}

response = requests.post(
    f'{API_ENDPOINT}/records/{record_id}/draft/actions/publish',
    headers=headers,
    cookies=cookies
)
print('POST publish:', response.json())